In [29]:
import pandas as pd
import googlemaps
import os
import numpy as np
#Get API Key from TXT file -- in gitignore
apikey = 'apikey.txt'



In [31]:
#Return contents of a file
def get_file_contents(filename):
    """ Given a filename,
        return the contents of that file
    """
    try:
        with open(filename, 'r') as f:
            # It's assumed our file contains a single line,
            # with our API key
            return f.read().strip()
    except FileNotFoundError:
        print("'%s' file not found" % filename)
api_key = get_file_contents(apikey)

In [32]:
#Run Google Maps
gmaps = googlemaps.Client(key=api_key)

In [33]:
#Import files

#First NEA files, of which there are multiple
files = os.listdir('NEA Grants')
the_list = []


for f in files:
    print(f)
    data = pd.read_excel('NEA Grants/' + f)
    the_list.append(data)

NEA = pd.concat(the_list)
NEH = pd.read_excel('NEHAwardSearchResults.xlsx')

2021.xlsx


/Users/kristenvinh/anaconda3/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


2020.xlsx


/Users/kristenvinh/anaconda3/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


2024.xlsx


/Users/kristenvinh/anaconda3/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


2023.xlsx


/Users/kristenvinh/anaconda3/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


2022.xlsx


/Users/kristenvinh/anaconda3/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [34]:
#Delete Rows with nan values
NEA = NEA.dropna(how='all', inplace=False)
NEH = NEH.dropna(how='all', inplace=False)

In [35]:
#Delete Cognressional District col so it's not confused with later imports
del NEA['Congressional District']

In [36]:
#Exporting Cleaned Full Grant list
NEA.to_csv("NEA_Full_Grant.csv")
NEH.to_csv("NEH_Full_Grant.csv")

In [37]:
#Get only columns only needed for Lat/Long Look-Up
NEA_Orgs = NEA[['Organization Name','City', 'State','Zip']]
NEH_Orgs = NEH[['Organization', 'Organization City', 'Organization State',
       'Organization Postal Code',]]

In [38]:
#Drop Duplicates
NEA_Orgs = NEA_Orgs.drop_duplicates()
NEH_Orgs = NEH_Orgs.drop_duplicates()

In [39]:
#Create full address column for lookup
NEA_Orgs['Full Address'] = NEA_Orgs['Organization Name'] + ", " + NEA_Orgs['City'] + ", " + NEA_Orgs['State'] + " " +str(NEA_Orgs['Zip'])
NEH_Orgs['Full Address'] = NEH_Orgs['Organization'] + ", " + NEH_Orgs['Organization City'] + ", " + NEH_Orgs['Organization State'] + " " + str(NEH_Orgs['Organization Postal Code'])

In [40]:
#Check size of NEA_Orgs
NEA_Orgs.shape

(5751, 5)

In [41]:
#Check size of NEH_Orgs
NEH_Orgs.shape

(1745, 5)

In [42]:
#Split Dataframes in manageable chunks to not exceed api limits
NEA_Orgs1, NEA_Orgs2, NEA_Orgs3, NEA_Orgs4, NEA_Orgs5 = np.array_split(NEA_Orgs, 5)
NEH_Orgs1, NEH_Orgs2 = np.array_split(NEH_Orgs, 2)

NEA_List = [NEA_Orgs1, NEA_Orgs2, NEA_Orgs3, NEA_Orgs4, NEA_Orgs5]
NEH_List = [NEH_Orgs1, NEH_Orgs2]


In [43]:
#Function to run geocoding
def geocoding(df):
    text_list = df["Full Address"].to_list()
    Lat_List = []
    Long_List = []
    for address in text_list:
        geo = gmaps.geocode(address)
        try: 
            lat = geo[0]['geometry']['bounds']['northeast']['lat']   
            lng = geo[0]['geometry']['bounds']['northeast']['lng']

        except:
            try: 
                lat = geo[0]['geometry']['location']['lat']   
                lng = geo[0]['geometry']['location']['lng'] 
            except:
                lat = "none"
                lng = "none"
        Lat_List.append(lat)
        Long_List.append(lng)
    df["Latitude"] = Lat_List
    df["Longitude"] = Long_List
    return df

In [49]:
#Running Geocoding and saving files
#Commenting out to avoid accidental runs NEH_List[0] = geocoding(NEH_List[0])
NEH_List[0]

,Organization,Organization City,Organization State,Organization Postal Code,Full Address,Latitude,Longitude
0,Tufts University,Somerville,MA,02144-2401,"Tufts University, Somerville, MA 0 02144...",42.418118,-71.072995
1,Northeastern Oklahoma State University,Tahlequah,OK,74464-2301,"Northeastern Oklahoma State University, Tahleq...",35.952788,-94.941901
2,Princeton University,Princeton,NJ,08540-5228,"Princeton University, Princeton, NJ 0 02...",40.391755,-74.617508
3,Goucher College,Baltimore,MD,21204-2753,"Goucher College, Baltimore, MD 0 02144-2...",39.372206,-76.529453
4,St. John Fisher College,Rochester,NY,14618-3537,"St. John Fisher College, Rochester, NY 0 ...",43.269045,-77.531166
...,...,...,...,...,...,...,...
1932,Putnam History Museum,Cold Spring,NY,10516-2613,"Putnam History Museum, Cold Spring, NY 0 ...",41.42539,-73.944864
1933,Frick Collection,New York,NY,10021-4981,"Frick Collection, New York, NY 0 02144-2...",40.917577,-73.700272
1934,Clark Art Institute,Williamstown,MA,01267-2878,"Clark Art Institute, Williamstown, MA 0 ...",42.74594,-73.163566
1935,Pratt Museum,Homer,AK,99603-7504,"Pratt Museum, Homer, AK 0 02144-2401\n1 ...",59.680543,-151.403995


In [51]:
#Commenting out to avoid accidental runs NEH_List[1] = geocoding(NEH_List[1])
NEH_List[1]

,Organization,Organization City,Organization State,Organization Postal Code,Full Address,Latitude,Longitude
1938,Burke Museum Association,Seattle,WA,98195-3010,"Burke Museum Association, Seattle, WA 0 ...",47.73528,-122.219822
1939,Currier Museum of Art,Manchester,NH,03104-4380,"Currier Museum of Art, Manchester, NH 0 ...",43.051474,-71.375409
1941,Katonah Museum of Art,Katonah,NY,10536-3736,"Katonah Museum of Art, Katonah, NY 0 021...",41.26629,-73.676391
1943,Baranov Museum,Kodiak,AK,99615-6307,"Baranov Museum, Kodiak, AK 0 02144-2401\...",57.813494,-152.355668
1944,Chicago Architecture Foundation,Chicago,IL,60604-2505,"Chicago Architecture Foundation, Chicago, IL 0...",42.023131,-87.523661
...,...,...,...,...,...,...,...
4137,Opera Lafayette,Washington,DC,20003-2141,"Opera Lafayette, Washington, DC 0 02144-...",38.995864,-76.909393
4140,American Philosophical Association,Newark,DE,19716-4200,"American Philosophical Association, Newark, DE...",39.714926,-75.723779
4142,University Press of Colorado,Louisville,CO,80027-1697,"University Press of Colorado, Louisville, CO 0...",39.997869,-105.107848
4145,Montana Department of Commerce,Helena,MT,59601-6282,"Montana Department of Commerce, Helena, MT 0 ...",46.627493,-111.948106


In [55]:
#Concating files back, exporting
NEH_With_Geo = pd.concat(NEH_List)
NEH_With_Geo.to_csv('NEH Org list with Geocoding.csv', index=False)
NEH_With_Geo

,Organization,Organization City,Organization State,Organization Postal Code,Full Address,Latitude,Longitude
0,Tufts University,Somerville,MA,02144-2401,"Tufts University, Somerville, MA 0 02144...",42.418118,-71.072995
1,Northeastern Oklahoma State University,Tahlequah,OK,74464-2301,"Northeastern Oklahoma State University, Tahleq...",35.952788,-94.941901
2,Princeton University,Princeton,NJ,08540-5228,"Princeton University, Princeton, NJ 0 02...",40.391755,-74.617508
3,Goucher College,Baltimore,MD,21204-2753,"Goucher College, Baltimore, MD 0 02144-2...",39.372206,-76.529453
4,St. John Fisher College,Rochester,NY,14618-3537,"St. John Fisher College, Rochester, NY 0 ...",43.269045,-77.531166
...,...,...,...,...,...,...,...
4137,Opera Lafayette,Washington,DC,20003-2141,"Opera Lafayette, Washington, DC 0 02144-...",38.995864,-76.909393
4140,American Philosophical Association,Newark,DE,19716-4200,"American Philosophical Association, Newark, DE...",39.714926,-75.723779
4142,University Press of Colorado,Louisville,CO,80027-1697,"University Press of Colorado, Louisville, CO 0...",39.997869,-105.107848
4145,Montana Department of Commerce,Helena,MT,59601-6282,"Montana Department of Commerce, Helena, MT 0 ...",46.627493,-111.948106


In [ ]:
#Running NEA Geocoding and saving files
NEA_Orgs.to_csv('NEA Org list with Geocoding.csv', index='False')